## TF-Faster-RCNN-RPN

----------------------
I will implement RPN network in this notebook. 


#### Objective of this notebook 
1. Trian a rpn for object localization 


### Section list

1. Define a generator that return (1, height, width, 3), (1, nb_boxes, 4), (1, nb_boxes, 1)
2. Define the conv. layers 
3. Define the rpn layers 
4. Define the loss tensors
5. build network 
6. Train the network 

### Improt dependency  

In [1]:
from tensorflow.contrib import layers
from tensorflow.contrib.framework.python.ops import arg_scope
from tensorflow.contrib.layers.python.layers import layers as layers_lib
from tensorflow.contrib.layers.python.layers import regularizers
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import variable_scope
import tensorflow.contrib.slim as slim
import tensorflow as tf
import os

from scipy.misc import imresize

from math import floor,exp
import pprint

import matplotlib.image as mpimg
import numpy as np
import matplotlib.pylab as plt
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import sklearn

### Define the global varables

In [2]:
anchor_box_scales = [128, 256, 512]
anchor_box_ratio = [[1,1],[1,2],[2,1]]
nb_anchors = len(anchor_box_scales) * len(anchor_box_ratio)

dataSets = ['VOC2012']


TEST_FULL_IMG = np.array([mpimg.imread("./test1.jpg")])
print(TEST_FULL_IMG.shape)
print(TEST_FULL_IMG.shape[1]/4, TEST_FULL_IMG.shape[2]/4)

(1, 720, 1280, 3)
180.0 320.0


## Define generator

We use VOC datasets in this implmentation.

### Get annotations 

In [3]:

def get_voc_data(input_path):
    all_imgs = []

    classes_count = {}

    class_mappingNameToId = {}
    class_mappingIdToName = {}

    visualise = False

    data_paths = [os.path.join(input_path, s) for s in dataSets]

    print ('Parsing annotation files')

    for data_path in data_paths:

        annot_path = os.path.join(data_path, 'Annotations')
        imgs_path = os.path.join(data_path, 'JPEGImages')
        imgsets_path_trainval = os.path.join(data_path, 'ImageSets',
                'Main', 'trainval.txt')

# ........imgsets_path_test = os.path.join(data_path, 'ImageSets','Main','test.txt')

        trainval_files = []
        test_files = []
        try:
            with open(imgsets_path_trainval) as f:
                for line in f:
                    trainval_files.append(line.strip() + '.jpg')
        except Exception as e:

# ............with open(imgsets_path_test) as f:
# ................for line in f:
# ....................test_files.append(line.strip() + '.jpg')

            print (e)

        annots = [os.path.join(annot_path, s) for s in
                  os.listdir(annot_path)]
        idx = 0
        for annot in annots:
            try:
                idx += 1

                et = ET.parse(annot)
                element = et.getroot()

                element_objs = element.findall('object')
                element_filename = element.find('filename').text
                element_width = int(element.find('size').find('width'
                                    ).text)
                element_height = int(element.find('size').find('height'
                        ).text)

                if len(element_objs) > 0:
                    annotation_data = {
                        'filepath': os.path.join(imgs_path,
                                element_filename),
                        'width': element_width,
                        'height': element_height,
                        'bboxes': [],
                        }

                    if element_filename in trainval_files:
                        annotation_data['imageset'] = 'trainval'
                    elif element_filename in test_files:
                        annotation_data['imageset'] = 'test'
                    else:
                        annotation_data['imageset'] = 'trainval'

                for element_obj in element_objs:
                    class_name = element_obj.find('name').text
                    if class_name not in classes_count:
                        classes_count[class_name] = 1
                    else:
                        classes_count[class_name] += 1

                    if class_name not in class_mappingNameToId:
                        class_mappingNameToId[class_name] = \
                            len(class_mappingNameToId)
                        class_mappingIdToName[len(class_mappingNameToId)
                                - 1] = class_name

                    obj_bbox = element_obj.find('bndbox')
                    x1 = int(round(float(obj_bbox.find('xmin').text)))
                    y1 = int(round(float(obj_bbox.find('ymin').text)))
                    x2 = int(round(float(obj_bbox.find('xmax').text)))
                    y2 = int(round(float(obj_bbox.find('ymax').text)))
                    w = x2 - x1
                    h = y2 - y1
                    x = int(round(x1 + w / 2))
                    y = int(round(y1 + h / 2))
                    difficulty = int(element_obj.find('difficult'
                            ).text) == 1
                    annotation_data['bboxes'].append({
                        'class': class_name,
                        'x1': x1,
                        'x2': x2,
                        'y1': y1,
                        'y2': y2,
                        'difficult': difficulty,
                        'x': x,
                        'y': y,
                        'w': w,
                        'h': h,
                        })
                all_imgs.append(annotation_data)

                if visualise:
                    img = cv2.imread(annotation_data['filepath'])
                    for bbox in annotation_data['bboxes']:
                        cv2.rectangle(img, (bbox['x1'], bbox['y1']),
                                (bbox['x2'], bbox['y2']), (0, 0, 255))
                    cv2.imshow('img', img)
                    cv2.waitKey(0)
            except Exception as  e:

                print (e)
                continue

        # make if no bg in the className make bg class

        if 'bg' not in classes_count:
            print ('bg not in class')
            
            class0Name = class_mappingIdToName[0]
            class0NewId = len(class_mappingNameToId)
            
            
            classes_count['bg'] = 0
            class_mappingNameToId['bg'] = 0 
            class_mappingIdToName[0] = 'bg'
            
            class_mappingNameToId[class0Name] = class0NewId
            class_mappingIdToName[len(class_mappingNameToId)-1] = class0Name
            
            
        else:
            print ('bg in class')
            # if there are a bg class make it first index
            bgOldId = class_mappingNameToId['bg']
            #switch id bg to 0id class
            if bgOldId != 0 :
                class0Name = class_mappingIdToName[0]
                class0NewId = bgOldId
                
                class_mappingIdToName[0] = 'bg'
                class_mappingNameToId['bg'] = 0
                
                class_mappingIdToName[class0NewId] = class0Name
                class_mappingNameToId[class0Name] = class0NewId
            
    return (all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName)

In [4]:
class Data_source:
    def __init__(self):
        all_imgs, classes_count, class_mappingNameToId, class_mappingIdToName = get_voc_data("./data")
        self.all_imgs = all_imgs
        self.classes_count = classes_count
        self.class_mappingNameToId = class_mappingNameToId
        self.class_mappingIdToName = class_mappingIdToName
        
    def get_classId(self, class_name):
        return self.class_mappingNameToId[class_name]
    
    def get_generator(self):
        '''
            generator use in training
            imgs 
            x,y,w,h
            x1,y1,x2,y2
            class
            
            return (1, height, width, 3), (1, nb_boxes, 4), (1, nb_boxes, 4), (1, nb_boxes, 1)
        '''
        records = sklearn.utils.shuffle(self.all_imgs)
        for record in records:
            img = mpimg.imread(record["filepath"])
            
            box_xywh = []
            box_xyxy = []
            box_class = []
            
            for box in record["bboxes"]:
                box_xywh.append([box["x"], box["y"], box["w"], box["h"]])
                box_xyxy.append([box["x1"], box["y1"], box["x2"], box["y2"]])
                name = self.get_classId(box["class"])
                box_class.append([name])
        
            yield np.array([img]), np.array([box_xywh]), np.array([box_xyxy]), np.array([box_class])
    

In [5]:
datas = Data_source()
generator = datas.get_generator()

Parsing annotation files
bg not in class


In [6]:

img, xtwh,xxyy, cls = next(generator)

## Define the conv. layers

---------

I will define vgg16 conv layers. 

Vgg 16 used in this project, it use the pretrain network from imageNet. The network defined until conv5.

The vgg16 function accept iamge input and otput the nets tensor and the enpoints.

We use conv5 as our feature map layer, which will port to rpn and rcnn.

In [7]:
# Define the vgg16 layers
def vgg_16(inputs,  scope='vgg_16'):
    with tf.variable_scope(scope, 'vgg_16', [inputs]) as sc:
        end_points_collection = sc.name + '_end_points'

        # Collect outputs for conv2d, fully_connected and max_pool2d.
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
#             net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
#             net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return net, end_points

## Define the rpn layers


we will define the RPN network. 

It have few steps. 
1. Define the conv layers of rpn.
2. Mapping rpn to bbox

In [30]:

def rpn_cls_shape_func(in_list):
    cls = in_list
    return np.array([cls.shape[0], cls.shape[1],cls.shape[2],cls.shape[3],1]).astype(np.int32)

def rpn_regr_shape_func(in_list):
    regr = in_list
    return np.array([regr.shape[0], regr.shape[1],regr.shape[2],regr.shape[3]/4,4]).astype(np.int32)

def rpn(net, num_anchors=9, scope="rpn"):
    with tf.variable_scope(scope, 'rpn', [net]) as sc:
        end_points_collection = sc.name + '_end_points'
        
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d], 
                            outputs_collections=end_points_collection, 
                            activation_fn=tf.nn.relu,
                            weights_initializer=tf.truncated_normal_initializer(0.0, 0.01)):
            
            net = slim.conv2d(net, 512, [3, 1], scope='rpn_conv_3x3', padding='SAME')
            
            rpn_class = slim.conv2d(net, num_anchors, [1, 1], scope='rpn_class')
            rpn_cls_shape = tf.py_func(rpn_cls_shape_func, [rpn_class], tf.int32, name="rpn_cls_shape")
            rpn_class = tf.reshape(rpn_class, rpn_cls_shape)
            
            rpn_regr = slim.conv2d(net, num_anchors*4, [1, 1], scope='rpn_regr')  
            rpn_regr_shape = tf.py_func(rpn_regr_shape_func, [rpn_regr], tf.int32, name="rpn_regr_shape")
            rpn_regr = tf.reshape(rpn_regr, rpn_regr_shape)

            # Convert end_points_collection into a end_point dict.
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
        
    return rpn_class, rpn_regr, end_points

#### Generate ground truth

In [129]:
def generate_anchor_regr_ground_truth(featureMap, img, bboxes, cls):
    '''
        for anchor regr ground truth it use parameterizations which is normalized
        return (1, featureMapRow, featureMapCrol, nb_anchor, 4), (1, featureMapRow, featureMapCol, nb_anchor, 1)
    '''
    stepSize = img[1]/featureMap[1]
    
    
    pass

def get_cls_ground_truth(res):
    return res[1]

def get_regr_ground_truth(res):
    return res[0]


def create_generate_anchors( anchor_box_scales = [128, 256, 512],   anchor_box_ratio = [[1,1],[1,2],[2,1]] ):
    def generate_anchors(featureMap, img):
        '''
            Not batch is return, which can save memory in run time 
        '''
        stepSize = int(img.shape[1]/featureMap.shape[1])
        imgWidth = img.shape[2]
        imgheight = img.shape[1]
        
        anchors = []
        for scale in anchor_box_scales:
            for ratio in anchor_box_ratio:
                anchors.append([ratio[0]*scale,ratio[1]*scale])
        anchors = np.array(anchors)

        bbox = np.zeros((180, 320, len(anchors), 4))

        x = range(int(stepSize/2), imgWidth, stepSize)
        y = range(int(stepSize/2), imgheight, stepSize)

        xv, yv= np.meshgrid(x, y)

        for anchorIdx, width, height in zip(range(len(anchors)), anchors[:,0], anchors[:,1]):
            bbox[:,:,anchorIdx,0] = xv
            bbox[:,:,anchorIdx,1] = yv
            bbox[:,:,anchorIdx,2].fill(width)
            bbox[:,:,anchorIdx,3].fill(height)
        bbox = bbox.astype(np.int32)

        return bbox
    return generate_anchors

def create_rpn_regr_ground_truth(anchors, bboxes):
    '''
        Since it is training one img is accepted 
        return t_x, t_y, t_w, t_h
    '''
#     res = np.copy(anchors).astype(np.float32)
    res = np.zeros(anchors.shape)
    

In [130]:
stepSize = 4
imgWidth = 1280
imgheight = 720
anchor_box_scales = [128, 256, 512]
anchor_box_ratio = [[1,1],[1,2],[2,1]]


anchors = []
for scale in anchor_box_scales:
    for ratio in anchor_box_ratio:
        anchors.append([ratio[0]*scale,ratio[1]*scale])
anchors = np.array(anchors)

bbox = np.zeros((180, 320, len(anchors), 4))

x = range(int(stepSize/2), imgWidth, stepSize)
y = range(int(stepSize/2), imgheight, stepSize)

xv, yv= np.meshgrid(x, y)

for anchorIdx, width, height in zip(range(len(anchors)), anchors[:,0], anchors[:,1]):
    bbox[:,:,anchorIdx,0] = xv
    bbox[:,:,anchorIdx,1] = yv
    bbox[:,:,anchorIdx,2].fill(width)
    bbox[:,:,anchorIdx,3].fill(height)
bbox = bbox.astype(np.int32)

In [131]:
bbox[0,1,2,:]

array([  6,   2, 256, 128], dtype=int32)

In [132]:
res.shape

(115200,)

## Define the loss tensors


In [133]:
def smooth_l1(bbox_pred, bbox_targets, name=""):
    """
        ResultLoss = outside_weights * SmoothL1(inside_weights * (bbox_pred - bbox_targets))
        SmoothL1(x) = 0.5 * ( x)^2,    if |x| < 1
                      |x| - 0.5 ,    otherwise
    """
    with tf.variable_scope("smooth_l1_"+name):
        x = tf.subtract(bbox_pred, bbox_targets)

        smooth_l1_sign = tf.cast(tf.less(tf.abs(x), 1.0 ), tf.float32)
        smooth_l1_option1 = tf.multiply(tf.multiply(x, x), 0.5 )
        smooth_l1_option2 = tf.subtract(tf.abs(x), 0.5 )
        smooth_l1_result = tf.add(tf.multiply(smooth_l1_option1, smooth_l1_sign),
                                  tf.multiply(smooth_l1_option2, tf.abs(tf.subtract(smooth_l1_sign, 1.0))))

    return smooth_l1_result

def rpn_loss():
    
    pass

## Build network

In [134]:

img_input = tf.placeholder(tf.float32, [1, None, None, 3])

# input the bbox location and the class id 
# in format x,y,w,h,classId
y_bbox_regr = tf.placeholder(tf.float32, [1, None, 4])

y_bbox_cls = tf.placeholder(tf.float32, [1, None, 1])

conv_layer,conv_end_points = vgg_16(img_input)
conv_restore_names = [ item for item in conv_end_points] 

rpn_class, rpn_regr, rpn_end_points = rpn(conv_layer,nb_anchors)
rpn_class_softmax = tf.nn.softmax(rpn_class)

generate_anchors = tf.py_func(create_generate_anchors(), [rpn_regr, img_input], tf.int32, name="generate_anchors")

# rpn_regr_ground_truth = 

# rpn_loss_regr = rpn_regr_loss(rpn_regr, rpn_y_bbox_res)
# rpn_loss_cls = rpn_class_loss(rpn_class, rpn_y_cls_res)
# rpn_loss = rpn_loss_regr + rpn_loss_cls

# rpn_optimizer = tf.train.AdamOptimizer(0.005).minimize(rpn_loss)


# restore weights
variables_to_restore = slim.get_variables_to_restore(include=conv_restore_names)
vgg_checkpoint_path = os.path.join("./", 'vgg_16.ckpt')
restorer = tf.train.Saver(variables_to_restore)


init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    # Restore variables from disk.
    restorer.restore(sess, "./vgg_16.ckpt")
    
    sess.run(init_op)
    print("restore conv layers")
    
    target_tensor = [conv_layer, rpn_class_softmax, rpn_regr,generate_anchors]
    
    res, rpn_class_res, rpn_regr_res, anchors = sess.run(target_tensor, feed_dict={img_input:TEST_FULL_IMG})
        

restore conv layers


TypeError: 'float' object cannot be interpreted as an integer

InternalError: Failed to run py callback pyfunc_17: see error log.
	 [[Node: generate_anchors_2 = PyFunc[Tin=[DT_FLOAT, DT_FLOAT], Tout=[DT_INT32], token="pyfunc_17", _device="/job:localhost/replica:0/task:0/cpu:0"](rpn_9/Reshape_1/_59, _recv_Placeholder_27_0)]]

Caused by op 'generate_anchors_2', defined at:
  File "/home/holman/anaconda2/envs/carND/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/holman/anaconda2/envs/carND/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/holman/anaconda2/envs/carND/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/holman/anaconda2/envs/carND/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/holman/anaconda2/envs/carND/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  

In [ ]:
rpn_class_res.shape

In [ ]:
rpn_regr_res.shape